<a href="https://colab.research.google.com/github/GuJiMao/GNN_Learning/blob/main/%E6%B6%88%E6%81%AF%E4%BC%A0%E9%80%92%E6%96%B9%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 说明
[原文链接](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_gnn.html#implementing-the-edge-convolution)

# 创建消息传递网络

将卷积算子推广到不规则域通常表示为邻域聚合或消息传递方案。其中的$x_i^{(k-1)}\in \mathbb{R}^F$表示的在第$(k-1)层的$节点$i$的节点特征，$e_{j,i}\in \mathbb{R}^D$表示的是从节点$j$到节点$i$的边的特征，图神经网络的消息传递机制可以被描述为：

$$\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right),$$

其中的$\square$表示的是一个可微分的，置换不变量函数（permutation invariant function），例如$\sum$, $\max$。$\gamma, \phi$表示可微分函数，例如MLPs（多层感知机）

## `MessagePassing`这个基类
PyTorch Geometric提供了`MessagePassing`这个基类，通过自动地消息传播实现图网络中的消息传递。

使用者们只需要简单地定义函数$\phi$，例如`message()`和$\gamma$，例如`update()`和要使用的聚合方法，例如`aggr="add"`， `aggr="mean"`，`aggr="max"`。

通过以下方式的帮助就能够完成上述内容：
- `MessagePassing(aggr="add", flow="source_to_target", node_dim=2)`：定义了聚类方案（`"add"`，`"mean"`，`"max"`）和消息的传播方向(`source_to_target`或者`target_to_source`)。另外，`node_dim`这个属性指明了要沿着哪个轴进行传播。
- `MessagePassing.propagate(edge_index, size=None, **kwargs)`：最初的传播消息的函数，接收边缘索引和所有额外的数据，这些数据是构造消息和更新节点嵌入所需的。需要注意`propagate()`并不限制于对称的邻接矩阵（大小为`[N,N]`）的消息传递中,但也可以在一般稀疏赋值矩阵中交换消息,例如二部置换图（通过传递大小为`size=(N,M)`作为额外的参数。）如果设置为`None`的话，假设赋值矩阵是对称的。对于具有两个独立的节点集和索引集的二部图，并且每个集持有自己的信息，这种分裂可以通过传递信息为元组来标记，例如`x=(x_N,x_M)`。
- `MessagePassing.message(...)`对于节点$i$中的每条边构造消息（类似于$\phi$）。如果消息是流向节点$i$的（`flow=source_to_target`），那么$(j,i)\in \mathcal{E}$；如果消息是流向节点$j$的（即`flow=target_to_source`），那么$(i,j)\in \matchal{E}$。可以接受最初传递给`propagate()`的任何参数。另外，传递给`propagate()`的tensor可以被映射到节点$i$和节点$j$上，只需要在变量后面加上`_i`或`_j`即可。注意：我们上述提到的$i$作为传递消息的中心节点，$j$作为邻节点，这是约定俗成的习惯。
- `MessagePassing.update(aggr_out, ...)`：更新节点的嵌入，对于每个节点$i\in \mathcal{V}$类似于$\gamma$。接受聚类输出作为第一个参数，和最初传递个`propagate()`的任意参数。

让我们通过重新实现[Kipf和Welling的GCN层](https://arxiv.org/abs/1609.02907)和[Wang等人的EdgeConv层](https://arxiv.org/abs/1801.07829)这两个流行的GNN变体来验证这一点。

## 实现GCN层
GCN层的数学描述为：
$$\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right),$$
其中邻节点特征首先通过一个权重矩阵$\Theta$被第一次改变，归一化他们的角度，最后归纳。这个公式可以被分为下列的步骤：
1. 在邻接矩阵中添加自循环。
2. 线性变换节点特征矩阵。
3. 计算归一化系数。
4. 归一化$\phi$中的节点特征
5. 把邻边节点特征相加(`aggr="add"`)

步骤1-3通常是在消息传递之前计算的。步骤4-5可以使用`messageppassing`基类轻松处理。完整的层实现如下所示

In [ ]:
import torch 
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(GCNConv, self).__init__(aggr='add')   # 添加步骤5中的聚类函数add
    self.lin = torch.nn.Linear(in_channels, out_channels)

  def forward(self, x, edge_index):
    # x的大小[N, in_channels]
    # 边索引的大小[2, E]

    # 步骤1：添加自循环到邻接矩阵
    edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

    # 步骤2：线性地变换特征矩阵
    x = self.lin(x)

    # 步骤3：计算归一化
    row, col = edge_index
    deg = degree(col, x.size(0), dtype=x.dtype)
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row]*deg_inv_sqrt[col]

    # 步骤4-5：开始传播消息
    return self.propagate(edge_index, x=x, norm=norm)
  def message(self, x_j, norm):
    # x_j的大小[E, out_channels]

    # 步骤4：归一化节点特征
    return norm.view(-1, 1)*x_j

`GCNConv`继承自带有`"add"`的传播参数的`MessagePassing`基类。层中的逻辑代替了其中的`forward`函数。这里，我们首先使用`torch_geometric.utils.add_self_loops()`将自循环添加到边索引中，通过调用`torch.nn.Linear`这个实例实现了线性变换。

对于节点$i$的归一化因子是从该节点的节点度数$deg(i)$得到的，对于$(j,i)\in \mathcal{E}$的边这个归一化因子就成为了$\frac{1}{\sqrt{deg(i)}\sqrt{deg(j)}}$。这个结果的大小为`[num_edges,]`保存在了tensor向量`norm`中。

在`message()`函数中，我们通过`norm`对相邻节点特征`x_j`进行规范化。其中，`x_j`表示一个提升张量，它包含了每条边的源节点特征，即每个节点的邻居。节点特性可以通过在变量名后面附加`_i`或`_j`来自动解除。事实上，任何张量都可以通过这种方式转换，只要它们包含源节点或目标节点特征。

这就是创建一个简单的消息传递层所需要的全部内容。您可以使用这一层作为深层架构的构建块。初始化和调用它很简单:

In [ ]:
conv = GCNConv(16, 32)
x = conv(x, edge_index)

## 实现边的卷积

[边缘卷积层](https://arxiv.org/abs/1801.07829)处理图形或点云，数学上定义为：
$$\mathbf{x}_i^{(k)} = \max_{j \in \mathcal{N}(i)} h_{\mathbf{\Theta}} \left( \mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)} - \mathbf{x}_i^{(k-1)} \right),$$

其中的$h_{\Theta}$表示一个MLP。类比于GCN层的实现，我们可以使用`MessagePassing`这个基类实现这一层，这次使用的为`aggr="max"`：

In [ ]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
  def __init__(self, in_channels, out_channels):
    super(EdgeConv, self).__init__(aggr='max')    # "max"传播
    self.mlp = Seq(Linear(2*in_channels, out_channels), ReLU(), Linear(out_channels, out_channels))

  def forward(self, x, edge_index):
    # x的大小为 [N, in_channels]
    # edge_index的大小[2, E]
    return self.propagate(edge_index, x=x)

  def message(self, x_i, x_j):
    # x_i大小[E, in_channels]
    # x_j大小[E, in_channels]

    tmp = torch.cat([x_i, x_j-x_i], dim=1)    # tmp的大小[E, 2*in_channels]
    return self.mlp(tmp)


在`message()`函数中，使用`self.mlp`来同时转换目标节点的特征和对于$(j,i)\in\mathcal{E}$源节点的特征`x_j-x_i`。边的卷积事实上是一个动态的卷积，使用特征空间中最近的邻居重新计算每一层的图形。PyTorch Geometric有一个名为`torch_geometric.nn.pool.knn_graph`的函数能够加速这个过程，因为它可以调用GPU。

In [ ]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super(DynamicEdgeConv, self).__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super(DynamicEdgeConv, self).forward(x, edge_index)

上面的`knn_graph()`计算最近邻的图，相当于使用`EdgeConv`中的方法`forward`。下面，我们就能够使用一个接口来实现这层

In [ ]:
conv = DynamicEdgeConv(3, 128, k=6)
x = conv(x, batch)